In [2]:
# Install required libraries
!pip install -q pandas numpy bokeh pathway

# Imports
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot
from math import radians, cos, sin, asin, sqrt
import pathway as pw
import time
import threading

# Enable Bokeh for Colab
output_notebook()

# ============================
# LOAD + PREPROCESS DATASET
# ============================
df = pd.read_csv("dataset.csv")
df.columns = df.columns.str.lower().str.strip()
column_mapping = {
    'id': 'record_id',
    'systemcodenumber': 'lot_id',
    'capacity': 'capacity',
    'latitude': 'latitude',
    'longitude': 'longitude',
    'occupancy': 'occupancy',
    'vehicletype': 'vehicle_type',
    'trafficconditionnearby': 'traffic_level',
    'queuelength': 'queue_length',
    'isspecialday': 'is_special_day',
    'lastupdateddate': 'date',
    'lastupdatedtime': 'time'
}
df = df.rename(columns=column_mapping)
df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time'], dayfirst=True, errors='coerce')
df = df.drop(columns=['date', 'time', 'record_id'])
df = df.sort_values('timestamp')

# Clean + Normalize
df['occupancy'] = pd.to_numeric(df['occupancy'], errors='coerce').fillna(0).astype(int)
df['capacity'] = pd.to_numeric(df['capacity'], errors='coerce').fillna(1).astype(int)
df['queue_length'] = pd.to_numeric(df['queue_length'], errors='coerce').fillna(0).astype(int)
df['is_special_day'] = pd.to_numeric(df['is_special_day'], errors='coerce').fillna(0).astype(int)
df['traffic_level'] = df['traffic_level'].astype(str).str.lower()
df['vehicle_type'] = df['vehicle_type'].astype(str).str.lower()

# Feature Engineering
vehicle_weights = {'car': 1, 'bike': 0.6, 'truck': 1.5}
df['occupancy_rate'] = df['occupancy'] / df['capacity'].replace(0, 1)
df['vehicle_weight'] = df['vehicle_type'].map(vehicle_weights).fillna(1)
df['normalized_traffic'] = df['traffic_level'].map({'low': 0.2, 'average': 0.5, 'high': 0.8}).fillna(0.5)

# ============================
# PRICING MODELS
# ============================

def model_1_linear(df, alpha=5):
    df['price_model1'] = 10 + alpha * df['occupancy_rate']
    return df['price_model1'].clip(lower=5, upper=20)

def compute_demand(row):
    return (
        1 * row['occupancy_rate'] +
        0.5 * row['queue_length'] -
        0.7 * row['normalized_traffic'] +
        1.0 * row['is_special_day'] +
        1.2 * row['vehicle_weight']
    )

def model_2_demand(df):
    df['demand'] = df.apply(compute_demand, axis=1)
    df['normalized_demand'] = (df['demand'] - df['demand'].mean()) / df['demand'].std()
    df['price_model2'] = 10 * (1 + 0.2 * df['normalized_demand'])
    return df['price_model2'].clip(lower=5, upper=20)

def vectorized_haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def model_3_competition(df):
    df['price_model3'] = df['price_model2'].copy()
    for ts in df['timestamp'].unique():
        ts_df = df[df['timestamp'] == ts]
        if len(ts_df) <= 1: continue
        lat, lon = ts_df['latitude'].values, ts_df['longitude'].values
        dist_matrix = np.zeros((len(ts_df), len(ts_df)))
        for i in range(len(ts_df)):
            dist_matrix[i] = vectorized_haversine(lat[i], lon[i], lat, lon)
        mask = (dist_matrix < 1.0) & (dist_matrix > 0)
        for i, idx in enumerate(ts_df.index):
            neighbors = np.where(mask[i])[0]
            if len(neighbors) > 0:
                avg_price = ts_df.iloc[neighbors]['price_model2'].mean()
                if ts_df.iloc[i]['occupancy_rate'] > 0.95 and avg_price < ts_df.iloc[i]['price_model2']:
                    df.at[idx, 'price_model3'] = max(5, ts_df.iloc[i]['price_model2'] - 1)
                elif avg_price > ts_df.iloc[i]['price_model2']:
                    df.at[idx, 'price_model3'] = min(20, ts_df.iloc[i]['price_model2'] + 1)
    return df['price_model3'].clip(lower=5, upper=20)

df['price_model1'] = model_1_linear(df)
df['price_model2'] = model_2_demand(df)
df['price_model3'] = model_3_competition(df)

# ============================
# REROUTING FUNCTION
# ============================
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return R * 2 * asin(sqrt(a))

def suggest_rerouting(df, lot_id, timestamp):
    lot_df = df[(df['lot_id'] == lot_id) & (df['timestamp'] == pd.to_datetime(timestamp))]
    if lot_df.empty or lot_df.iloc[0]['occupancy_rate'] <= 0.95:
        return None
    row = lot_df.iloc[0]
    nearby = df[(df['timestamp'] == row['timestamp']) & (df['lot_id'] != lot_id)].copy()
    nearby['distance'] = nearby.apply(
        lambda r: haversine(row['latitude'], row['longitude'], r['latitude'], r['longitude']),
        axis=1
    )
    alternatives = nearby[(nearby['distance'] < 1.0) & (nearby['occupancy_rate'] < 0.8)]
    if not alternatives.empty:
        best = alternatives.sort_values('price_model3').iloc[0]
        return f"Reroute to {best['lot_id']} (Price: ${best['price_model3']:.2f}, Distance: {best['distance']:.2f} km)"
    return None

# ============================
# SAVE CLEANED DATA
# ============================
df.to_csv('modified_dataset.csv', index=False)

# ============================
# PATHWAY STREAMING SETUP
# ============================
class ParkingSchema(pw.Schema):
    timestamp: str
    lot_id: str
    occupancy: int
    capacity: int
    queue_length: int
    traffic_level: str
    is_special_day: int
    vehicle_type: str
    latitude: float
    longitude: float
    occupancy_rate: float
    vehicle_weight: float
    normalized_traffic: float
    price_model1: float
    price_model2: float
    price_model3: float

def simulate_streaming(input_csv, output_csv, delay_seconds=0.01):
    df = pd.read_csv(input_csv)
    df['timestamp'] = df['timestamp'].astype(str)
    with open(output_csv, 'w') as f:
        f.write(','.join(df.columns) + '\n')
    for _, row in df.iterrows():
        with open(output_csv, 'a') as f:
            f.write(','.join(map(str, row.values)) + '\n')
        time.sleep(delay_seconds)

# Start streaming
threading.Thread(target=simulate_streaming, args=('modified_dataset.csv', 'streaming_dataset.csv')).start()

# Run Pathway pipeline
table = pw.io.csv.read('streaming_dataset.csv', schema=ParkingSchema, mode='streaming')
avg_prices = table.groupby(table.lot_id).reduce(
    lot_id=table.lot_id,
    avg_price_model1=pw.reducers.avg(table.price_model1),
    avg_price_model2=pw.reducers.avg(table.price_model2),
    avg_price_model3=pw.reducers.avg(table.price_model3)
)
pw.io.csv.write(avg_prices, 'output_prices.csv')
pw.run(monitoring_level=pw.MonitoringLevel.ALL)

# ============================
# VISUALIZATION
# ============================
def plot_comparison(df, lot_id):
    df_lot = df[df['lot_id'] == lot_id]
    source = ColumnDataSource(df_lot)
    p = figure(title=f"Pricing Models - {lot_id}", x_axis_type="datetime", width=400, height=400)
    p.line('timestamp', 'price_model1', source=source, color="blue", legend_label="Model 1")
    p.line('timestamp', 'price_model2', source=source, color="green", legend_label="Model 2")
    p.line('timestamp', 'price_model3', source=source, color="red", legend_label="Model 3")
    p.legend.location = "top_left"
    return p

def plot_competitor_comparison(df, lot_id, timestamp):
    lot_df = df[(df['lot_id'] == lot_id) & (df['timestamp'] == pd.to_datetime(timestamp))]
    if lot_df.empty: return None
    row = lot_df.iloc[0]
    nearby = df[(df['timestamp'] == row['timestamp']) & (df['lot_id'] != lot_id)].copy()
    nearby['distance'] = nearby.apply(lambda r: haversine(row['latitude'], row['longitude'], r['latitude'], r['longitude']), axis=1)
    competitors = nearby[nearby['distance'] < 1.0]
    if competitors.empty: return None
    data = {'lot_id': [lot_id] + competitors['lot_id'].tolist(), 'price': [row['price_model3']] + competitors['price_model2'].tolist()}
    p = figure(x_range=data['lot_id'], title=f"Competitor Prices - {lot_id} @ {timestamp}", width=400, height=400)
    p.vbar(x='lot_id', top='price', source=ColumnDataSource(pd.DataFrame(data)), width=0.4, color="purple")
    return p

# Show plots
p1 = plot_comparison(df, 'BHMBCCMKT01')
p2 = plot_competitor_comparison(df, 'BHMBCCMKT01', '2016-12-18 08:30:00')
show(gridplot([[p1, p2]]))

# ============================
# REAL-TIME PREDICTIONS
# ============================
print("🚗 Real-Time Pricing Predictions:\n")
for _, row in df.iterrows():
    lot_id, timestamp = row['lot_id'], row['timestamp']
    print(f"[{timestamp}] Lot {lot_id} | M1=${row['price_model1']:.2f} M2=${row['price_model2']:.2f} M3=${row['price_model3']:.2f}")
    reroute = suggest_rerouting(df, lot_id, timestamp)
    if reroute: print(f"🔁 {reroute}")


FileNotFoundError: [Errno 2] No such file or directory: 'dataset.csv'